# Offline Batch Recommender System

In this notebook, we will build a simple offline batch recsys that writes recommendations to Redis for quick retrieval later. The architecture diagram below shows how it comes together from a bird's eye view.

![](./img/OfflineBatchRecsys.png)

## Architecture
Recommender systems commonly have a *multi-stage pipeline*:
1) A fast **Candidate Retrieval Model** quickly truncates the large item catalog to a relevant set of hundreds (or thousands) of items.
2) Filtering is performed to remove undesirable or already-seen items.
3) A finely-tuned deep learning **Ranking Model** (i.e. more powerful) ranks the most likely items that are going to interacted with.
4) Results are ordered and returned to the user.

In this case, we will write recommendations to a key-value store (Redis) such that a client can request these in near "realtime" at a later point.

> WHY? This is especially useful for developers who can't afford the complexity of hosting a live multi-stage recsys OR need to get something up and running *quickly*

In this notebook, we will:

1) Prepare the [**Dataset**]()
2) Build a [**Candidate Retrieval Model**](#candidate)
3) Build a [**Ranking Model**]()
4) [**Write Recommendations**]() to Redis (offline)
5) [**Fetch Recommendations**]() from Redis
6) [**Export Models**]() for later use

*This notebook was created using the latest stable [merlin-tensorflow](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/merlin/containers/merlin-tensorflow/tags) container and was heavily based on the work done by the NVIDIA Merlin team [here](https://github.com/NVIDIA-Merlin/models/blob/main/examples/05-Retrieval-Model.ipynb)*

## Dataset Preparation

We will use a synthetic dataset that mimicks the [Ali-CCP: Alibaba Click and Conversion Prediction](https://tianchi.aliyun.com/dataset/dataDetail?dataId=408#1) dataset. This allows us to tune it to our exact needs for demonstration/learning purposes.


### Importing Libraries

In [1]:
import os
import logging

import nvtabular as nvt
import merlin.models.tf as mm
import tensorflow as tf

from nvtabular.ops import *

from merlin.datasets.synthetic import generate_data
from merlin.datasets.ecommerce import transform_aliccp
from merlin.models.utils.example_utils import workflow_fit_transform
from merlin.models.utils.dataset import unique_rows_by_features
from merlin.schema.tags import Tags
from merlin.io.dataset import Dataset


# disable INFO and DEBUG logging everywhere
logging.disable(logging.WARNING)

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-01-12 15:50:23.617092: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2023-01-12 15:50:28.778743: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-12 15

### Generate Synthetic Ali-CCP Dataset

In [2]:
# Generate the synthetic data

NUM_ROWS = 1000000
TRAIN_SIZE = 0.7
VALID_SIZE = 0.3

train, valid = generate_data("aliccp-raw", NUM_ROWS, set_sizes=(TRAIN_SIZE, VALID_SIZE))

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


**Note:** To be able to learn from this implicit feedback, we use the naive assumption that the interacted items are **more relevant** for the user than the non-interacted ones.

This is an assumption for simplification purposes so we can use a negative sampling technique. 

In [3]:
# Defin output path for data
DATA_DIR = os.environ['PWD'] +"/data/"
OUTPUT_DATA_DIR = os.path.join(DATA_DIR, "processed")
OUTPUT_RETRIEVAL_DATA_DIR = os.path.join(OUTPUT_DATA_DIR, "retrieval")
CATEGORY_TEMP_DIR = os.path.join(DATA_DIR, "categories")

In [4]:
# Define Feature Transformation Pipeline

user_id_raw = ["user_id"] >> Rename(postfix='_raw') >> LambdaOp(lambda col: col.astype("int32")) >> TagAsUserFeatures()
item_id_raw = ["item_id"] >> Rename(postfix='_raw') >> LambdaOp(lambda col: col.astype("int32")) >> TagAsItemFeatures()

user_id = ["user_id"] >> Categorify(dtype="int32", out_path=CATEGORY_TEMP_DIR) >> TagAsUserID()
item_id = ["item_id"] >> Categorify(dtype="int32", out_path=CATEGORY_TEMP_DIR) >> TagAsItemID()

item_features = (
    ["item_category", "item_shop", "item_brand"] >> Categorify(dtype="int32", out_path=CATEGORY_TEMP_DIR) >> TagAsItemFeatures()
)

user_features = (
    [
        "user_shops",
        "user_profile",
        "user_group",
        "user_gender",
        "user_age",
        "user_consumption_2",
        "user_is_occupied",
        "user_geography",
        "user_intentions",
        "user_brands",
        "user_categories",
    ]
    >> Categorify(dtype="int32", out_path=CATEGORY_TEMP_DIR)
    >> TagAsUserFeatures()
)

targets = ["click"] >> AddMetadata(tags=[Tags.BINARY_CLASSIFICATION, "target"])

outputs = user_id + item_id + item_features + user_features +  user_id_raw + item_id_raw + targets

# add dropna op to filter rows with nulls
outputs = outputs >> Dropna()

With `transform_aliccp` function, we can execute fit() and transform() on the raw dataset applying the operators defined in the NVTabular workflow pipeline above. The processed parquet files are saved to output_path.

In [7]:
# Transform data and create files
transform_aliccp((train, valid), OUTPUT_DATA_DIR, nvt_workflow=outputs)

## Candidate Retrieval Model

We will use a **Two-Tower** model to infer a subset of relevant items from large item corpus for a given user. 

A Two-Tower Model consists of item (candidate) and user (query) encoder towers. With two towers, the model can learn representations (embeddings) for queries and candidates separately. 

> NEED TO FIND IMG
<img src="./images/TwoTower.png"  width="30%">

Image Adapted from: [Off-policy Learning in Two-stage Recommender Systems](https://dl.acm.org/doi/abs/10.1145/3366423.3380130)

### Feature Engineering for Candidate Retrieval

In [8]:
# Load Datasets from generated files
train_retrieval = Dataset(os.path.join(OUTPUT_DATA_DIR, "train", "*.parquet"))
valid_retrieval = Dataset(os.path.join(OUTPUT_DATA_DIR, "valid", "*.parquet"))valid_retrieval

In [10]:
# NVTabular Feature Transformations

inputs = train_retrieval.schema.column_names

# Select only positive interaction rows where click==1 in the dataset with Filter() operator

outputs = inputs >> Filter(f=lambda df: df["click"] == 1)

workflow = nvt.Workflow(outputs)

workflow.fit(train_retrieval)

workflow.transform(train_retrieval).to_parquet(
    output_path=os.path.join(OUTPUT_RETRIEVAL_DATA_DIR, "train")
)

workflow.transform(valid_retrieval).to_parquet(
    output_path=os.path.join(OUTPUT_RETRIEVAL_DATA_DIR, "valid")
)

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


*NVTabular exported the schema file, schema.pbtxt a protobuf text file, of our processed dataset. To learn more about the schema object and schema file you can explore [this notebook](https://github.com/NVIDIA-Merlin/models/blob/main/examples/02-Merlin-Models-and-NVTabular-integration.ipynb).*

In [13]:
# Read filtered parquet files as Dataset objects

train_retrieval = Dataset(os.path.join(OUTPUT_RETRIEVAL_DATA_DIR, "train", "*.parquet"), part_size="500MB")
valid_retrieval = Dataset(os.path.join(OUTPUT_RETRIEVAL_DATA_DIR, "valid", "*.parquet"), part_size="500MB")

Now we can use the `schema` object to define the model inputs. We select features with user and item tags, and exclude target column.

In [15]:
schema = train_retrieval.schema.select_by_tag([Tags.ITEM_ID, Tags.USER_ID, Tags.ITEM, Tags.USER]).without(['user_id_raw', 'item_id_raw', 'click'])
train_retrieval.schema = schema
valid_retrieval.schema = schema

In [16]:
# Inspect the schema!

schema

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.start_index,properties.cat_path,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.domain.min,properties.domain.max,properties.domain.name
0,user_id,"(Tags.CATEGORICAL, Tags.USER, Tags.USER_ID, Ta...",int32,False,False,None,0.0,0.0,0.0,/workdir/data/processed/workflow/categories/un...,768.0,66.0,0,767,user_id
1,item_id,"(Tags.CATEGORICAL, Tags.ITEM, Tags.ITEM_ID, Ta...",int32,False,False,None,0.0,0.0,0.0,/workdir/data/processed/workflow/categories/un...,763.0,66.0,0,762,item_id
2,item_category,"(Tags.CATEGORICAL, Tags.ITEM)",int32,False,False,None,0.0,0.0,0.0,/workdir/data/processed/workflow/categories/un...,763.0,66.0,0,762,item_category
3,item_shop,"(Tags.CATEGORICAL, Tags.ITEM)",int32,False,False,None,0.0,0.0,0.0,/workdir/data/processed/workflow/categories/un...,763.0,66.0,0,762,item_shop
4,item_brand,"(Tags.CATEGORICAL, Tags.ITEM)",int32,False,False,None,0.0,0.0,0.0,/workdir/data/processed/workflow/categories/un...,763.0,66.0,0,762,item_brand
5,user_shops,"(Tags.CATEGORICAL, Tags.USER)",int32,False,False,None,0.0,0.0,0.0,/workdir/data/processed/workflow/categories/un...,768.0,66.0,0,767,user_shops
6,user_profile,"(Tags.CATEGORICAL, Tags.USER)",int32,False,False,None,0.0,0.0,0.0,/workdir/data/processed/workflow/categories/un...,42.0,16.0,0,41,user_profile
7,user_group,"(Tags.CATEGORICAL, Tags.USER)",int32,False,False,None,0.0,0.0,0.0,/workdir/data/processed/workflow/categories/un...,10.0,16.0,0,9,user_group
8,user_gender,"(Tags.CATEGORICAL, Tags.USER)",int32,False,False,None,0.0,0.0,0.0,/workdir/data/processed/workflow/categories/un...,3.0,16.0,0,2,user_gender
9,user_age,"(Tags.CATEGORICAL, Tags.USER)",int32,False,False,None,0.0,0.0,0.0,/workdir/data/processed/workflow/categories/un...,7.0,16.0,0,6,user_age


As expected, we shouldn't have any label/target data yet

In [44]:
label_names = schema.select_by_tag(Tags.TARGET).column_names
label_names

[]

### Designing Retrieval Model Architecture

The **Two-Tower** model consists of a **User tower** (where all user features are fed) and an **Item tower** (where all item features are fed).

The User tower generates an embedding for the User. Then it computes the positive interaction "score" (likelihood of interaction event) using the dot-product between the User embedding and the Item embedding, in addition to sampled "negative" Items within a batch.

##### About Negative Sampling

Many datasets for recommender systems contain implicit feedback with logs of user interactions like clicks, add-to-cart, purchases, music listening events, rather than explicit ratings that reflects user preferences over items. 

In Merlin Models -- NVIDIA provides some scalable negative sampling algorithms for this Item Retrieval task. In this example, we use the `in-batch` sampling algorithm which uses the items interacted by other users as negatives within the same mini-batch.

In [31]:
# Function to create the two tower retrieval model

def create_two_tower(tower_dim: int, encoder_dim: int, optimizer: str, k: int, tags) -> mm.TwoTowerModelV2:
    # User/Query Tower
    user_schema = schema.select_by_tag(tags.USER)
    # create user (query) tower input block
    user_inputs = mm.InputBlockV2(user_schema)
    # create user (query) encoder block
    query = mm.Encoder(
        user_inputs,
        mm.MLPBlock([encoder_dim, tower_dim], no_activation_last_layer=True)
    )

    # Item/Candidate Tower
    item_schema = schema.select_by_tag(tags.ITEM)
    # create item (candidate) tower input block
    item_inputs = mm.InputBlockV2(item_schema)
    # create item (candidate) encoder block
    candidate = mm.Encoder(
        item_inputs,
        mm.MLPBlock([encoder_dim, tower_dim], no_activation_last_layer=True)
    )
    
    # Build Model Class
    model = mm.TwoTowerModelV2(query, candidate)
    model.compile(
        optimizer=optimizer,
        run_eagerly=False,
        loss="categorical_crossentropy",
        metrics=[mm.RecallAt(k), mm.NDCGAt(k)]
    )
    return model

**Notes:**
- `no_activation_last_layer:` when set True, no activation is used for top hidden layer. Learn more [here](https://storage.googleapis.com/pub-tools-public-publication-data/pdf/b9f4e78a8830fe5afcf2f0452862fb3c0d6584ea.pdf).
- In the `TwoTowerModelV2` function we did not set `negative_samplers` arg. By default, it uses contrastive learning and `in-batch` negative sampling strategy.
- Two metrics are used to judge the quality of the recommendations: **Normalized Discounted Cumulative Gain (NDCG@K)** and **Recall@K**.
    - NDCG@K accounts for rank of the relevant item in the recommendation list and is a more fine-grained metric than HR, which only verifies whether the relevant item is among the top-k items.
    - Recall (Also known as HitRate@K) when there is only one relevant item in the recommendation list. Recall just verifies whether the relevant item is among the top-k items.
- When we set `validation_data=valid` in the `model.fit()`, we compute evaluation metrics on validation set using the negative sampling strategy used for training. 

In [19]:
# Initialize model
retrieval_model = create_two_tower(
    tower_dim=64,
    encoder_dim=128,
    optimizer="adam",
    k=10,
    tags=Tags
)

# Fit model
retrieval_model.fit(train_retrieval, validation_data=valid_retrieval, batch_size=4096, epochs=2)

Epoch 1/2
86/86 [==============================] - 9s 60ms/step - loss: 8.2954 - recall_at_10: 0.0251 - ndcg_at_10: 0.0234 - regularization_loss: 0.0000e+00 - loss_batch: 8.2701 - val_loss: 8.2923 - val_recall_at_10: 0.0306 - val_ndcg_at_10: 0.0306 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 7.9081
Epoch 2/2
86/86 [==============================] - 4s 49ms/step - loss: 8.2948 - recall_at_10: 0.0300 - ndcg_at_10: 0.0298 - regularization_loss: 0.0000e+00 - loss_batch: 8.2695 - val_loss: 8.2928 - val_recall_at_10: 0.0170 - val_ndcg_at_10: 0.0156 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 7.9098


### Evaluate the model accuracy

The validation metric values during training are calculated given the positive and negative scores in each batch, and then averaged over batches per epoch. **That means validation metrics are not computed using the entire item catalog.**

To determine the exact accuracy, we need to compute the similarity score between a given query and all possible candidates. Below, by using the `topk_model` we can evaluate the trained retrieval model using the entire item catalog (brute force).

In [20]:
# Create candidate/item features for evaluation
candidate_features = unique_rows_by_features(train_retrieval, Tags.ITEM, Tags.ITEM_ID)

In [21]:
# Here's a display of all of the items
candidate_features.to_ddf().compute()

,item_id,item_category,item_shop,item_brand
5,1,1,1,1
43,2,2,2,2
2,3,3,3,3
85,4,4,4,4
7,5,5,5,5
...,...,...,...,...
178438,753,753,753,753
58858,754,754,754,754
215811,756,756,756,756
119888,757,757,757,757


In [22]:
# Convert model to a top_k_encoder
topk_model = retrieval_model.to_top_k_encoder(candidate_features, k=20, batch_size=128)

# we can set `metrics` param in the `compile(), if we want
topk_model.compile(run_eagerly=False)

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


In [23]:
# Create data loader for validation data
eval_loader = mm.Loader(valid_retrieval, batch_size=1024).map(mm.ToTarget(schema, "item_id"))

# Evaluation
metrics = topk_model.evaluate(eval_loader, return_dict=True)
metrics

147/147 [==============================] - 4s 14ms/step - loss: 0.3710 - recall_at_10: 0.0824 - mrr_at_10: 0.0307 - ndcg_at_10: 0.0427 - map_at_10: 0.0307 - precision_at_10: 0.0082 - regularization_loss: 0.0000e+00 - loss_batch: 0.3707


{'loss': 0.3710492253303528,
 'recall_at_10': 0.08240917325019836,
 'mrr_at_10': 0.030697820708155632,
 'ndcg_at_10': 0.04271082207560539,
 'map_at_10': 0.030697820708155632,
 'precision_at_10': 0.008240917697548866,
 'regularization_loss': 0.0,
 'loss_batch': 0.3309170603752136}

### Generate top-K recommendations

Let's generate top-K (k=20 in our example) recommendations for a given batch of 8 samples. The `to_top_k_encoder()` method uses the item/candidate features dataset to compute and store all item/candidate embeddings in an index. The forward method of `topk_model` takes as the query/user features as input, and computes the dot product scores between the given query/user embeddings and all the candidates of the top-k index. Then, it returns the top-k (k=20) item ids with the highest scores.

In [25]:
# Create query/user features for evaluation

user_features = unique_rows_by_features(train_retrieval, Tags.USER, Tags.USER_ID)

In [26]:
user_features.to_ddf().compute()

,user_id,user_shops,user_profile,user_group,user_gender,user_age,user_consumption_2,user_is_occupied,user_geography,user_intentions,user_brands,user_categories
17,1,1,1,1,1,1,1,1,1,1,1,1
55,2,2,1,1,1,1,1,1,1,2,2,2
18,3,3,1,1,1,1,1,1,1,3,3,3
24,4,4,1,1,1,1,1,1,1,4,4,4
117,5,5,1,1,1,1,1,1,1,5,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...
264149,763,763,37,5,1,3,2,1,2,763,763,763
192329,764,764,38,6,1,3,2,1,2,764,764,764
270949,765,765,39,7,2,4,2,2,4,765,765,765
271913,766,766,40,8,2,5,3,2,3,766,766,766


In [27]:
# Check out one batch of users
loader = mm.Loader(user_features, batch_size=8, shuffle=False)
batch = next(iter(loader))
print(batch[0]['user_id'])

tf.Tensor(
[[1]
 [2]
 [3]
 [4]
 [5]
 [6]
 [7]
 [8]], shape=(8, 1), dtype=int32)


The recommended top 20 item ids are returned below for each of the 8 selected users (from the validation set). The output of the method is a named tuple `TopKPrediction`, where the first element is the dot product scores and the second element is the encoded item ids (not the original ids).

In [28]:
scores, reccommended_item_ids = topk_model(batch[0])

In [29]:
# Recommended Items for the batch of 8 Users
reccommended_item_ids

<tf.Tensor: shape=(8, 20), dtype=int32, numpy=
array([[760, 163,  61,  29, 220, 256,  27, 317,  21, 251, 339, 224, 176,
         14, 109,  12,  36, 515, 247, 482],
       [ 43, 233,   1,  95, 313,  33, 102,   8, 140,  34, 314, 545,   5,
        485, 521,  13, 685, 446, 591, 462],
       [  5,   8, 301,  34,   4, 357, 213, 140, 257, 453, 567,  14, 591,
        612, 125, 273,  97,   2,  43, 496],
       [ 18,  11, 255,  86,  50,   7,  61, 408,  12, 535, 290, 389, 159,
        115,  96, 355, 283,  73, 650,  44],
       [  5,  68,   4, 446, 485, 212, 388, 314, 273, 609, 710, 612,   8,
         15, 321, 567, 451, 394, 591, 233],
       [  5,  18,  50,  11, 119, 215, 280, 138, 441,   1, 350,  45,  49,
         48, 267, 232, 384, 171,  77, 446],
       [ 50,  18,  11, 171,  28, 370,  74, 119,  96, 404,  65, 350, 103,
        603,   5, 212, 267, 441, 271, 705],
       [117,  12,   7, 199, 257, 273,  38, 120,  11, 612,  83,   5, 103,
        189, 250, 290,  28,   8, 603,  70]], dtype=int32)>

In [30]:
# Recommendation "scores" (higher is better) for each item in topK for each User in the batch of 8
scores

<tf.Tensor: shape=(8, 20), dtype=float32, numpy=
array([[0.07148756, 0.0592928 , 0.04263241, 0.04206377, 0.04134098,
        0.04126939, 0.03950732, 0.03849062, 0.03766272, 0.03731737,
        0.03611607, 0.03563635, 0.03438456, 0.03436154, 0.03428886,
        0.03335116, 0.03275171, 0.03250426, 0.03107942, 0.0306492 ],
       [0.05896503, 0.0550338 , 0.05237116, 0.05209834, 0.05191278,
        0.04809685, 0.0416616 , 0.04103876, 0.04036508, 0.04025036,
        0.03998354, 0.03993648, 0.03984115, 0.03883475, 0.03769161,
        0.03723952, 0.03710738, 0.03707945, 0.03631726, 0.03543824],
       [0.10223916, 0.07071984, 0.05490056, 0.05185218, 0.0435054 ,
        0.04271939, 0.03781777, 0.0372151 , 0.03473343, 0.03246345,
        0.03245043, 0.03241496, 0.03152925, 0.03114765, 0.03059477,
        0.02976292, 0.02886857, 0.02860777, 0.02751431, 0.02698508],
       [0.04644962, 0.04082624, 0.04082165, 0.03851781, 0.03596479,
        0.0353808 , 0.03353062, 0.03244458, 0.03243339, 0.030840

## Ranking Model

We will use a Deep Learning Recommendation Model [(DLRM)](https://arxiv.org/abs/1906.00091) architecture (published by Facebook/Meta in 2019) to score and rank User/Item pairs. The model was introduced as a personalization deep learning model that uses embeddings to process sparse features that represent categorical data and a multilayer perceptron (MLP) to process dense features, then interacts these features explicitly using the statistical techniques proposed in [here](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=5694074). To learn more about DLRM architetcture please visit [this notebook](https://github.com/NVIDIA-Merlin/models/blob/main/examples/04-Exporting-ranking-models.ipynb) in the Merlin Models GH repo.


### Feature Engineering for DLRM

In [35]:
# Define train and valid dataset objects
train_rank = Dataset(os.path.join(OUTPUT_DATA_DIR, "train", "*.parquet"), part_size="500MB")
valid_rank = Dataset(os.path.join(OUTPUT_DATA_DIR, "valid", "*.parquet"), part_size="500MB")

# Define schema object
schema = train_rank.schema.without(['user_id_raw', 'item_id_raw'])

In [36]:
# Inspect schema - DLRM takes all of these features as input!
schema

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.start_index,properties.cat_path,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.domain.min,properties.domain.max,properties.domain.name
0,user_id,"(Tags.CATEGORICAL, Tags.USER, Tags.USER_ID, Ta...",int32,False,False,NaN,0.0,0.0,0.0,/workdir/data/processed/workflow/categories/un...,768.0,66.0,0,767,user_id
1,item_id,"(Tags.CATEGORICAL, Tags.ITEM, Tags.ITEM_ID, Ta...",int32,False,False,NaN,0.0,0.0,0.0,/workdir/data/processed/workflow/categories/un...,763.0,66.0,0,762,item_id
2,item_category,"(Tags.CATEGORICAL, Tags.ITEM)",int32,False,False,NaN,0.0,0.0,0.0,/workdir/data/processed/workflow/categories/un...,763.0,66.0,0,762,item_category
3,item_shop,"(Tags.CATEGORICAL, Tags.ITEM)",int32,False,False,NaN,0.0,0.0,0.0,/workdir/data/processed/workflow/categories/un...,763.0,66.0,0,762,item_shop
4,item_brand,"(Tags.CATEGORICAL, Tags.ITEM)",int32,False,False,NaN,0.0,0.0,0.0,/workdir/data/processed/workflow/categories/un...,763.0,66.0,0,762,item_brand
5,user_shops,"(Tags.CATEGORICAL, Tags.USER)",int32,False,False,NaN,0.0,0.0,0.0,/workdir/data/processed/workflow/categories/un...,768.0,66.0,0,767,user_shops
6,user_profile,"(Tags.CATEGORICAL, Tags.USER)",int32,False,False,NaN,0.0,0.0,0.0,/workdir/data/processed/workflow/categories/un...,42.0,16.0,0,41,user_profile
7,user_group,"(Tags.CATEGORICAL, Tags.USER)",int32,False,False,NaN,0.0,0.0,0.0,/workdir/data/processed/workflow/categories/un...,10.0,16.0,0,9,user_group
8,user_gender,"(Tags.CATEGORICAL, Tags.USER)",int32,False,False,NaN,0.0,0.0,0.0,/workdir/data/processed/workflow/categories/un...,3.0,16.0,0,2,user_gender
9,user_age,"(Tags.CATEGORICAL, Tags.USER)",int32,False,False,NaN,0.0,0.0,0.0,/workdir/data/processed/workflow/categories/un...,7.0,16.0,0,6,user_age


In [34]:
# Target column here is "click" (binary classification)
target_column = schema.select_by_tag(Tags.TARGET).column_names[0]
target_column

'click'

### Building DLRM

In [39]:
def create_dlrm(optimizer: str, schema, target_column: str, embedding_dim: int):
    model = mm.DLRMModel(
        schema,
        embedding_dim=embedding_dim,
        bottom_block=mm.MLPBlock([128, 64]),
        top_block=mm.MLPBlock([128, 64, 32]),
        prediction_tasks=mm.BinaryClassificationTask(target_column),
    )
    model.compile(optimizer=optimizer, run_eagerly=False, metrics=[tf.keras.metrics.AUC()])
    return model


In [53]:
dlrm = create_dlrm(
    optimizer="adam",
    schema=schema,
    target_column=target_column,
    embedding_dim=64
)

dlrm.fit(train_rank, validation_data=valid_rank, batch_size=16 * 1024, epochs=3)

Epoch 1/3
43/43 [==============================] - 5s 56ms/step - loss: 0.6932 - auc_1: 0.4998 - regularization_loss: 0.0000e+00 - loss_batch: 0.6932 - val_loss: 0.6931 - val_auc_1: 0.5000 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 0.6931
Epoch 2/3
43/43 [==============================] - 1s 24ms/step - loss: 0.6931 - auc_1: 0.5062 - regularization_loss: 0.0000e+00 - loss_batch: 0.6931 - val_loss: 0.6932 - val_auc_1: 0.5010 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 0.6933
Epoch 3/3
43/43 [==============================] - 1s 24ms/step - loss: 0.6925 - auc_1: 0.5210 - regularization_loss: 0.0000e+00 - loss_batch: 0.6925 - val_loss: 0.6938 - val_auc_1: 0.5014 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 0.6940


In [54]:
# Check out one batch of users
loader = mm.Loader(valid_rank, batch_size=8, shuffle=False)
batch = next(iter(loader))
print(batch[0]['user_id'])

tf.Tensor(
[[ 39]
 [ 35]
 [ 76]
 [  6]
 [138]
 [  9]
 [ 12]
 [108]], shape=(8, 1), dtype=int32)


In [56]:
# Test the DLRM model I/O with the user batch
dlrm(batch[0])

<tf.Tensor: shape=(8, 1), dtype=float32, numpy=
array([[0.48006314],
       [0.52211267],
       [0.47032687],
       [0.53310156],
       [0.51482207],
       [0.49466288],
       [0.53760624],
       [0.50179195]], dtype=float32)>

### Test the entire pipeline

So now that we have a **Candidate Retrieval Model** and a **Ranking Model** (DLRM) we can test the end to end flow (locally). In order to do this live, like in the next multi-stage section, you need a Feature Store (Redis/Feast), VSS Index (Redis), and means to host/serve the entire DAG of operations (Triton).

Below is a very brute force approach for a single batch of users

In [215]:
# Test full pipeline
import numpy as np

BATCH_SIZE=8

# User and Item Feature "Stores"
item_features = unique_rows_by_features(train_retrieval, Tags.ITEM, Tags.ITEM_ID).to_ddf().compute()
user_features = unique_rows_by_features(train_retrieval, Tags.USER, Tags.USER_ID).to_ddf().compute()

# User batch loader
user_loader = mm.Loader(unique_rows_by_features(train_retrieval, Tags.USER, Tags.USER_ID), batch_size=BATCH_SIZE, shuffle=False)

# Sample batch
user_batch = next(iter(user_loader))
users = user_batch[0]['user_id']

In [216]:
# Retrieve candidates
_, candidate_item_ids = topk_model(user_batch[0])

for user, candidates in zip(users.numpy(), candidate_item_ids.numpy()):
    
    num_recs = len(candidates)
    user_id = user[0]
    
    # user features
    u_features = user_features[user_features.user_id == user_id]
    u_features = u_features.append([u_features]*(num_recs-1), ignore_index=True)


    # item features
    top_items = item_features[item_features.item_id.isin(candidates)].reset_index(drop=True)
    
    # combined features
    top_items[user_features.columns] = u_features
    top_items = Dataset(top_items)
    top_items.schema = schema.without(['click'])
    
    # Score with DLRM
    inputs = mm.Loader(top_items, batch_size=num_recs)
    inputs = next(iter(inputs))
    scores = dlrm(inputs[0]).numpy().reshape(-1)
    
    # Rank
    recs = candidates[np.where(scores > 0.5)].tolist()
    
    

In [217]:
recs

[117, 12, 7, 257, 273, 120, 11, 612, 5, 103, 189, 250, 603, 70]

## Write Recommendations to Redis

Redis is used (low latency k-v store) to persist recommendations for each User.


In [220]:
import asyncio
import redis.asyncio as redis

from redis.commands.json.path import Path


def generate_recs(topk_model, dlrm, input_data, batch_size: int, tags):
    user_loader = mm.Loader(unique_rows_by_features(input_data, tags.USER, tags.USER_ID), batch_size=batch_size, shuffle=False)
    # Load a batch
    for batch in loader:
        
        users = batch[0]['user_id']
        
        # Generate candidates per user
        _, candidate_item_ids = topk_model(batch[0])
        
        # Score candidates per user
        for user, candidates in zip(users.numpy(), candidate_item_ids.numpy()):

            num_recs = len(candidates)
            user_id = user[0]

            # user features
            u_features = user_features[user_features.user_id == user_id]
            u_features = u_features.append([u_features]*(num_recs-1), ignore_index=True)


            # item features
            top_items = item_features[item_features.item_id.isin(candidates)].reset_index(drop=True)

            # combined features
            top_items[user_features.columns] = u_features
            top_items = Dataset(top_items)
            top_items.schema = schema.without(['click'])

            # Score with DLRM
            inputs = mm.Loader(top_items, batch_size=num_recs)
            inputs = next(iter(inputs))
            scores = dlrm(inputs[0]).numpy().reshape(-1)

            # Rank
            recs = candidates[np.where(scores > 0.5)].tolist()
            
            yield user_id, recs

In [222]:
# Test Recommendation Generator
next(generate_recs(topk_model, dlrm, train_retrieval, 32, Tags))

# SEE BELOW: User ID --> Top K Item IDs

(39, [115, 18, 5, 50, 272, 370, 174, 58, 82, 710, 446, 179, 217])

In [223]:
async def store_recommendations(topk_model, dlrm, data_input, tags, n: int, redis_conn: redis.Redis):
    """
    Store recommendations generated for each User.
    """
    semaphore = asyncio.Semaphore(n)
    async def store(user_id: str, recs: list):
        """
        Store and individual User's latest recommendations in Redis.
        """
        async with semaphore:
            entry = {
                "user_id": int(user_id),
                "recommendations": [int(rec) for rec in recs]
            }
            # Set the JSON object in Redis
            await redis_conn.json().set(f"USER:{user_id}", Path.root_path(), entry)
    
    # create generator
    top_recs_per_user = generate_recs(topk_model, dlrm, data_input, 32, tags)
    # gather with "concurrency"
    await asyncio.gather(*[store(user_id, recs) for user_id, recs in top_recs_per_user])

In [226]:
redis_conn = redis.Redis(
    host="redis-inference-store",
    port=6379,
    decode_responses=True
)

# Run the process
await store_recommendations(topk_model, dlrm, train_retrieval, Tags, n=100, redis_conn=redis_conn)

/tmp/ipykernel_102/3240227347.py:21: RuntimeWarning: coroutine 'store_recommendations.<locals>.store' was never awaited
  await asyncio.gather(*[store(user_id, recs) for user_id, recs in top_recs_per_user])


ValueError: Column must have no nulls.

In [ ]:
!redis-cli -h redis-inference-store -p 6379 hgetall USER:1

## Conclusion

Now you have all of the tools to **train**

blah blah blah

## Exporting Models

So far we have trained and evaluated our Retrieval model. Now, the next step is to deploy our model and generate top-K recommendations given a user (query). We can efficiently serve our model by indexing the trained item embeddings into an **Approximate Nearest Neighbors (ANN)** engine. Basically, for a given user query vector, that is generated passing the user features into user tower of retrieval model, we do an ANN search query to find the ids of nearby item vectors, and at serve time, we score user embeddings over all indexed top-K item embeddings within the ANN engine.

In doing so, we need to export
 
- user (query) tower
- item and user features
- item embeddings

#### Save and Load User (query) tower

We are able to save the user tower model as a TF model to disk. The user tower model is needed to generate a user embedding vector when a user feature vector <i>x</i> is fed into that model.

In [167]:
query_tower = model.query_encoder
query_tower.save(os.path.join(DATA_DIR, "query_tower"))

## we can load back the saved model via the following script.
#query_tower_loaded = tf.keras.models.load_model(os.path.join(DATA_FOLDER, 'query_tower'))

#### Extract and save User features

With `unique_rows_by_features` utility function we can easily extract both unique user and item features tables as cuDF dataframes. Note that for user features table, we use `USER` and `USER_ID` tags.

In [168]:
user_features = (
    unique_rows_by_features(train, Tags.USER, Tags.USER_ID).compute().reset_index(drop=True)
)

In [169]:
user_features.head()

,user_id,user_shops,user_profile,user_group,user_gender,user_age,user_consumption_2,user_is_occupied,user_geography,user_intentions,user_brands,user_categories
0,1,1,1,1,1,1,1,1,1,1,1,1
1,2,2,1,1,1,1,1,1,1,2,2,2
2,3,3,1,1,1,1,1,1,1,3,3,3
3,4,4,1,1,1,1,1,1,1,4,4,4
4,5,5,1,1,1,1,1,1,1,5,5,5


In [171]:
# save to disk
user_features.to_parquet(os.path.join(DATA_DIR, "user_features.parquet"))

#### Generate Query embeddings for entire user catalog

In [172]:
queries = model.query_embeddings(Dataset(user_features, schema=schema), batch_size=1024, index=Tags.USER_ID)
query_embs_df = queries.compute(scheduler="synchronous").reset_index()

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(


In [173]:
query_embs_df.head()

,user_id,0,1,2,3,4,5,6,7,8,...,54,55,56,57,58,59,60,61,62,63
0,1,-0.044415,-0.079928,0.141791,-0.054618,0.071080,-0.129675,-0.022908,-0.098638,-0.165534,...,0.069087,-0.041935,-0.088211,0.009338,0.042672,0.064693,0.089439,-0.046101,0.019635,0.063172
1,2,0.050006,0.048904,0.122381,-0.045254,0.087561,-0.083287,0.047019,-0.062766,-0.025593,...,-0.034007,0.008304,0.003901,-0.041683,0.066743,0.026556,0.051612,-0.048407,0.101967,0.016460
2,3,0.032441,0.068938,0.144220,0.025580,-0.021241,-0.087041,-0.005831,-0.011544,-0.028608,...,-0.039868,0.022170,-0.067877,-0.045480,0.029166,0.085423,-0.011185,-0.077492,0.052504,0.085522
3,4,-0.000057,0.027755,0.110519,0.004052,0.026682,-0.023924,0.000899,-0.079054,-0.121010,...,0.042694,-0.068973,-0.055938,-0.039705,0.065993,-0.022096,0.046860,-0.075874,0.105141,0.074089
4,5,0.038788,0.093953,0.085692,-0.030298,0.056857,-0.076180,0.046644,-0.035246,-0.080857,...,-0.020631,-0.004258,0.006572,-0.032787,0.025109,-0.013951,-0.019807,-0.078905,0.125748,0.012741


#### Extract and save Item features

In [174]:
item_features = (
    unique_rows_by_features(train, Tags.ITEM, Tags.ITEM_ID).compute().reset_index(drop=True)
)

In [175]:
item_features.head()

,item_id,item_category,item_shop,item_brand
0,1,1,1,1
1,2,2,2,2
2,3,3,3,3
3,4,4,4,4
4,5,5,5,5


In [177]:
# save to disk
item_features.to_parquet(os.path.join(DATA_DIR, "item_features.parquet"))

#### Extract and save Item embeddings

In [178]:
item_embs = model.candidate_embeddings(Dataset(item_features, schema=schema), batch_size=1024, index=Tags.ITEM_ID)

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


In [179]:
item_embs_df = item_embs.compute(scheduler="synchronous")

In [180]:
item_embs_df

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
item_id,,,,,,,,,,,,,,,,,,,,,
1,-0.086588,-0.035170,0.038872,-0.030351,-0.005060,-0.026615,0.036075,-0.019011,-0.025655,0.061539,...,0.079430,-0.023591,0.039861,0.001400,0.047819,-0.046945,0.080522,-0.049821,0.011490,-0.034052
2,-0.020607,-0.028564,0.052904,-0.044290,0.044060,-0.077663,0.063523,-0.002706,-0.049997,-0.046659,...,-0.019446,0.017111,0.047408,0.014948,0.037164,-0.055093,0.080557,-0.004503,0.046443,-0.029852
3,0.014424,0.062466,0.026331,-0.016829,0.067588,-0.040853,0.049609,-0.068072,-0.040935,-0.016965,...,0.029904,0.049761,0.036740,-0.054622,0.017131,-0.079658,-0.063603,-0.020487,0.013681,0.013386
4,-0.100455,-0.087459,-0.002279,0.015967,-0.054807,0.021265,-0.003342,0.098855,-0.029481,0.007833,...,0.119475,0.023623,0.032304,-0.038052,0.066611,-0.063890,0.049579,-0.140459,0.029618,-0.030336
5,-0.010816,0.033060,0.055615,0.017870,0.008282,-0.040606,0.008903,-0.037174,-0.029034,0.012322,...,0.061865,-0.025471,0.040039,-0.002353,0.021088,-0.013709,0.019025,-0.025305,-0.004278,0.054150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668,-0.011978,0.008819,0.016523,-0.073915,0.054216,-0.067495,0.008583,-0.067215,0.014997,0.033973,...,0.052676,0.002763,0.043437,-0.001684,0.025481,-0.051444,0.030094,-0.034564,-0.005135,-0.015033
669,-0.051683,-0.021606,0.018060,-0.038789,-0.007095,-0.028825,0.026450,0.028714,-0.007321,0.013074,...,0.093724,0.010164,0.032347,-0.009498,-0.022861,-0.095878,0.034653,-0.067983,-0.050209,-0.018708
670,-0.014077,-0.005224,0.057278,-0.001175,-0.005469,0.012073,0.023982,-0.001092,-0.052760,0.024767,...,0.066937,-0.007940,0.025360,-0.028201,0.024175,-0.053981,0.062025,-0.054338,-0.030390,0.019311


In [182]:
# save to disk
item_embs_df.to_parquet(os.path.join(DATA_DIR, "item_embeddings.parquet"))

That's it. You have learned how to train and evaluate your Two-Tower retrieval model, and then how to export the required components to be able to deploy this model to generate recommendations. In order to learn more on serving a model to [Triton Inference Server](https://github.com/triton-inference-server/server), please explore the examples in the [Merlin](https://github.com/NVIDIA-Merlin/Merlin) and [Merlin Systems](https://github.com/NVIDIA-Merlin/systems) repos.